<a href="https://colab.research.google.com/github/codekennML/FakeNews-Detection/blob/main/Fake_News_Prediction_Stemmer_%26_tfidfvectorizer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Algorithm will aim to distinguish between news (Fake & Real) 
Using Tfidf Vectorizer

In [ ]:
from zipfile import ZipFile 
with ZipFile('/content/train.csv.zip', 'r') as zp:
 zp.extractall()
 print('File Extraction done')

File Extraction done


In [ ]:
#Dataset Attributes 

#  train.csv: A full training dataset with the following attributes:

#  id: unique id for a news article
#  title: the title of a news article
#  author: author of the news article
#  text: the text of the article; could be incomplete
#  label: a label that marks the article as potentially unreliable
#  1: unreliable
#  0: reliable

In [ ]:
# Importing Dependencies 

import  numpy as np 
import pandas as pd
import re 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 

print('Dependencies Loaded, LFG...')

Dependencies Loaded, LFG...


In [ ]:
# Download the nltk stopwords 
#Stopwords are characters or phrases that do not add or subtract extensive meaning from 
# a statement or sentence e.g "The, and , etc "

import nltk 
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Lets print the stopwords , in english 

print(stopwords.words('english'))

#As discussed earlier , the stopwords are seen below 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
#Lets do some data preprocessing 
#Load the data into a pandas dataframe

df =  pd.read_csv('/content/train.csv')

print('Dataset loaded sucessfully ')

Dataset loaded sucessfully 


In [ ]:
#Check the number of rows and columns 

df.shape

#The dataset has 20800 rows and 5 columns 

In [ ]:
#Lets visualize the columns 

df.head()

In [ ]:
#Lets check the number of missing values in the dataset 

df.isnull().sum()

#558 authors are missing , 1957 authors missing and 39 texts are missing 

#Since the number of missing values are a bit negliglibe and their disposal would not adversely affect the training 

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
#Lets replace the null values with an empty string 

df = df.fillna('')



In [ ]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
#A bit of feature selection - Merging title and author 
df['content'] = df['author']+ ' '+df['title']
print(df['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:
#Seperating the data into features and targets 
x = df.drop(['label'], axis =  1 )
y = df['label']



In [ ]:
#Now we proceed to stemming the data , Stemming is the process of reducing a word to its root 
# acting , actor -> act

porter_stem =  PorterStemmer()
  # defining the stemming function 

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',str(content))
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [porter_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content




In [ ]:
#Applying our just defined function to the content column of our df dataset

df['content'] =  df['content'].apply(stemming)

In [ ]:
#Print the stemmed content 
print(df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
x = df['content'].values
y =  df['label'].values

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv'] [1 0 1 ... 0 1 1]


In [ ]:
#Since our model cannot process text inputs
#We have to convert the textual data into numerical data using tfidfVectorizer

text_to_num =  TfidfVectorizer()
text_to_num.fit(x)

x_transfmd = text_to_num.transform(x)


In [ ]:
#print x 
print(x_transfmd)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [ ]:
#Splitting the dataset into training and test data 

x_train, x_test, y_train, y_test =  train_test_split(x_transfmd,y,test_size = 0.2,stratify = y, random_state = 2)

In [ ]:
#training the model 
LogReg =  LogisticRegression()

training_result  =  LogReg.fit(x_train,y_train)

In [ ]:
#Model evaluation 
y_predict =  LogReg.predict(x_test)
accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.9790865384615385


In [ ]:
#Making our prediction 

x_new =  x_test[0]
prediction = LogReg.predict(x_new)

if (prediction[0] == 0):
  print ('The News is Real ')
else: 
    print('Fake News!!!!!')


Fake News!!!!!
